In [ ]:
import json
import os

from dotenv import load_dotenv

from pydantic import BaseModel, ValidationError, Field
from typing import List

from openai import AsyncOpenAI

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

from semantic_kernel.functions import KernelArguments

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [ ]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [ ]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests"""

# Create the prompt execution settings and configure the Pydantic model response format
settings = OpenAIChatPromptExecutionSettings(response_format=TravelPlan)

agent = ChatCompletionAgent(
    service=chat_completion_service,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings) 
)

In [ ]:
from IPython.display import display, HTML


async def main():
    # Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    # Respond to user input
    user_inputs = [
        "Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne",
    ]

    for user_input in user_inputs:
        
        # Start building HTML output
        html_output = "<div style='margin-bottom:10px'>"
        html_output += "<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += "</div>"

        # Collect the agent's response
        response = await agent.get_response(messages=user_input, thread=thread)
        thread = response.thread

        try:
            # Try to validate the response as a TravelPlan
            travel_plan = TravelPlan.model_validate(json.loads(response.message.content))

            # Display the validated model as formatted JSON
            formatted_json = travel_plan.model_dump_json(indent=4)
            html_output += "<div style='margin-bottom:20px'>"
            html_output += "<div style='font-weight:bold'>Validated Travel Plan:</div>"
            html_output += f"<pre style='margin-left:20px; padding:10px; border-radius:5px;'>{formatted_json}</pre>"
            html_output += "</div>"
        except ValidationError as e:
            # Handle validation errors
            html_output += "<div style='margin-bottom:20px; color:red;'>"
            html_output += "<div style='font-weight:bold'>Validation Error:</div>"
            html_output += f"<pre style='margin-left:20px;'>{str(e)}</pre>"
            html_output += "</div>"
            # Add this to see what the response contains for debugging
            html_output += "<div style='margin-bottom:20px;'>"
            html_output += "<div style='font-weight:bold'>Raw Response:</div>"
            html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.content}</div>"
            html_output += "</div>"

        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()

물론입니다! 번역된 내용은 아래와 같습니다:

---

# 소개

이 문서는 @@INLINE_CODE_0@@를 사용하는 방법에 대한 기본적인 개요를 제공합니다. 이 도구는 개발자들이 더 효율적으로 작업할 수 있도록 설계되었습니다.

## 주요 기능

- **자동화된 프로세스**: 반복적인 작업을 줄여줍니다.
- **사용자 친화적인 인터페이스**: 직관적인 디자인으로 쉽게 사용할 수 있습니다.
- **확장 가능성**: 다양한 플러그인과 통합을 지원합니다.

## 설치

@@INLINE_CODE_1@@를 설치하려면 다음 단계를 따르세요:

1. @@INLINE_CODE_2@@를 다운로드합니다.
2. @@INLINE_CODE_3@@를 실행합니다.
3. 설치가 완료되면 @@INLINE_CODE_4@@를 확인하세요.

[!NOTE] 설치 과정에서 문제가 발생하면 공식 문서를 참조하세요.

## 사용법

### 기본 사용법

다음은 @@INLINE_CODE_5@@를 사용하는 기본적인 예제입니다:

```json
User:
Create a travel plan for a family of 4, with 2 kids, from Singapore to Melboune
Validated Travel Plan:
{
    "main_task": "Plan a family trip from Singapore to Melbourne for 4 people including 2 kids.",
    "subtasks": [
        {
            "assigned_agent": "FlightBooking",
            "task_details": "Book round-trip flights from Singapore to Melbourne for 2 adults and 2 children."
        },
        {
            "assigned_agent": "HotelBooking",
            "task_details": "Find and book a family-friendly hotel in Melbourne that accommodates 4 people."
        },
        {
            "assigned_agent": "CarRental",
            "task_details": "Arrange for a car rental in Melbourne suitable for a family of 4."
        },
        {
            "assigned_agent": "ActivitiesBooking",
            "task_details": "Plan and book family-friendly activities in Melbourne suitable for kids."
        },
        {
            "assigned_agent": "DestinationInfo",
            "task_details": "Provide information about Melbourne, including attractions, dining options, and family-oriented activities."
        }
    ]
}
```

위 코드는 간단한 작업을 수행하며, 더 복잡한 작업을 위해 확장할 수 있습니다.

[!TIP] 더 많은 예제를 보려면 공식 예제 페이지를 방문하세요.

### 고급 사용법

고급 기능을 사용하려면 다음을 참고하세요:

@@CODE_BLOCK_1@@

이 코드는 고급 설정을 보여주며, 사용자 정의가 가능합니다.

[!WARNING] 고급 설정을 변경하기 전에 반드시 백업을 만들어 두세요.

## 문제 해결

### 일반적인 문제

- **문제**: @@INLINE_CODE_6@@가 작동하지 않습니다.
  - **해결 방법**: @@INLINE_CODE_7@@를 확인하고 다시 시도하세요.

- **문제**: 설치 중 오류가 발생합니다.
  - **해결 방법**: 관리자 권한으로 실행하거나 @@INLINE_CODE_8@@를 확인하세요.

[!CAUTION] 문제를 해결할 수 없을 경우, 지원팀에 문의하세요.

## 결론

@@INLINE_CODE_9@@는 개발자들에게 강력한 도구를 제공합니다. 이 문서를 참고하여 시작해보세요!

[!IMPORTANT] 최신 업데이트를 확인하고 항상 최신 버전을 사용하세요.

---



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서를 해당 언어로 작성된 상태에서 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역을 사용함으로써 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
